<a href="https://colab.research.google.com/github/hocthucv/demo/blob/task1/location_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report: Running Data

In [222]:
# !pip install pykalman

In [223]:
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from pykalman import KalmanFilter

def map_plot_create(df, lat_field="lat", lon_field="lng"):
    df.dropna(
        axis=0,
        how='any',
        thresh=None,
        subset=None,
        inplace=True
    )

    color_scale = [(0, 'orange'), (1,'red')]

    fig = px.scatter_mapbox(df, 
                            lat=lat_field, 
                            lon=lon_field, 
                            color_continuous_scale=color_scale,
                            zoom=18, 
                            height=480,
                            width=800)

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    # fig.show()
    return fig

In [224]:
import requests
import json

url = "http://ai.pgo.sv0.console.papagroup.net:12001/datahub/get_location_data"

def get_location_data(runId, url=url):

    payload = json.dumps({
    "runId": runId
    })
    headers = {
    'Content-Type': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    data = pd.DataFrame(response.json()["data"]["coordinatesList"])
    return data

## Case1: Phone Lying On Table

In [225]:
lying_data = get_location_data(920)

plot = map_plot_create(lying_data)
plot.show()

## Case 2: Phone Shaking 

When shaking phone, the collected locations data contains a lot of noises

In [226]:
shaking_data = get_location_data(921)

plot = map_plot_create(shaking_data)
plot.show()

Apply noise filter by zscore

In [227]:
plot = map_plot_create(shaking_data[(np.abs(stats.zscore(shaking_data)) < 1).all(axis=1)])
plot.show()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [228]:
def apply_kf(df):
    measurements = np.asarray(df)
    kf = KalmanFilter(transition_matrices = [[1, 1], [0, 1]], observation_matrices = [[0.1, 0.5], [-0.3, 0.0]])
    kf = kf.em(measurements, n_iter=5)
    (filtered_state_means, filtered_state_covariances) = kf.filter(measurements)
    (smoothed_state_means, smoothed_state_covariances) = kf.smooth(measurements)
    return pd.DataFrame(filtered_state_means, columns =['lat', 'lng'])